## Загрузка данных по чугунам

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

df = pd.read_excel('нейросеть.xlsx')

In [ ]:
df["номер образца"] = df["номер образца"].fillna(method="ffill")

In [ ]:
# некоторые образцы нюансы типа скорости помечены в номер образца, сбросить те которые портят результаты

filter_specimen = [value for value in df["номер образца"].unique() if "V 3" in str(value)]
df = df[~df["номер образца"].isin(filter_specimen)]
filter_specimen = [value for value in df["номер образца"].unique() if "V 0,5" in str(value)]
df = df[~df["номер образца"].isin(filter_specimen)]
filter_specimen = [value for value in df["номер образца"].unique() if "V 0,9" in str(value)]
df = df[~df["номер образца"].isin(filter_specimen)]
drop_columns = [
    "номер образца", "Образец",
]
df = df.drop(columns=drop_columns)

In [ ]:
# В ходе работы обнаружили, что измерения с радиусом 115 / массой 430 были не на нежестких и таким образом оптимальные массу радиус не подобрать. 
# Нужно удалить массу и радиус лишние, и искать оптимальные параметры в других
df = df[df['mass'] != 0.004300]
indentor_drop_cols = ["mass", "r"]
df = df.drop(columns=indentor_drop_cols)

In [ ]:
y_df_hb = df["HB"]
x_df_hb = df.drop(columns="HB")

x_hb_train, x_hb_test, y_hb_train, y_hb_test = train_test_split(x_df_hb, y_df_hb, 
                                                train_size=0.8, 
                                                random_state=15)

In [ ]:
hb_predictor = LinearRegression(n_jobs=-1)
hb_predictor.fit(X=x_hb_train, y=y_hb_train)

predicted_hb = hb_predictor.predict(x_hb_test)
nevyazka_hb = hb_predictor.predict(x_hb_train)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_hb_test, predicted_hb))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_hb_test, predicted_hb))
plt.scatter(y_hb_train, nevyazka_hb)
plt.scatter(y_hb_test, predicted_hb)

plt.show()

In [ ]:
importance = hb_predictor.coef_
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

`Энергия деформирования Дж`,
`Энергия упругого деформирования Дж`,
`Энергия вязкого деформирования Дж`.
и на этом графике не видно, но при отсутствии энергий еще `e`

имеют максимальное влияние на результат. Что если их убрать?

Index(['расстояние от заделки', 'Жесткость по Маквеллу и Фойгту Н/м',
       'Вязкость по Фойгту Н*с/м', 'Вязкость по Максвеллу Н*с/м',
       'Тангенс потерь град', 'Энергия деформирования Дж',
       'Энергия упругого деформирования Дж',
       'Энергия вязкого деформирования Дж', 'Время удара мкс',
       'Максимальное усилие Н', 'Максимальное внедрение мкм', 'активное_время',
       'vmax', 'vmin', 'rigidity1', 'r', 'ost_def', 'a_p_pmax', 'p_a_amax',
       'ugol_naklona', 'modul_po_ugly', 'mass', 'внедр_при_макс_силе',
       'delta_force', 'угол_all', 'угол_50', 'угол_30', 'contact_depth',
       'contact_area', 'ugol_naklona_2', 'reduced_modulus', 'P / a', 'e',
       'Pmax/apmax', 'Pamax/amax', 'HB'],
      dtype='object')

In [ ]:
class_feature = 'расстояние от заделки'

In [ ]:
best_features = [
    "Энергия деформирования Дж",	
    "Энергия упругого деформирования Дж",	
    "Энергия вязкого деформирования Дж",
    "e",
    class_feature
]
df_bf = df[best_features + ["HB"]]
df_of = df.drop(columns=best_features)
df_bf.head()

In [ ]:
# split datasets by weird data clustering
filter_by = "e"
# filter_specimen = [value for value in df[filter_by].unique() if value < ]
df_bf_1 = df_bf[df_bf[filter_by] <= 0.001]
df_bf_2 = df_bf[df_bf[filter_by] > 0.001]


In [ ]:
# FULL
sns.pairplot(df_bf, hue=class_feature, markers=["o", "s", "D", "v", "^", "<", ">"]) # , markers=["o", "s", "D", "v", "^", "<", ">"]
plt.show()

In [ ]:
# HEAD
sns.pairplot(df_bf.head(592), hue=class_feature) # , markers=["o", "s", "D", "v", "^", "<", ">"]
plt.show()

In [ ]:
# TAIL
sns.pairplot(df_bf.tail(320), hue=class_feature, markers=["o", "s", "D", "v", "^", "<", ">"]) # , markers=["o", "s", "D", "v", "^", "<", ">"]
plt.show()

In [ ]:
# First cluster
sns.pairplot(df_bf_1, hue=class_feature) # , markers=["o", "s", "D", "v", "^", "<", ">"]
plt.show()

In [ ]:
# Second cluster
sns.pairplot(df_bf_2, hue=class_feature) # , markers=["o", "s", "D", "v", "^", "<", ">"]
plt.show()

In [ ]:
y_df_hb = df_of["HB"]
x_df_hb = df_of.drop(columns="HB")

x_hb_train, x_hb_test, y_hb_train, y_hb_test = train_test_split(x_df_hb, y_df_hb, 
                                                train_size=0.8, 
                                                random_state=18)


In [ ]:
hb_predictor = LinearRegression(n_jobs=-1)
hb_predictor.fit(X=x_hb_train, y=y_hb_train)

predicted_hb = hb_predictor.predict(x_hb_test)
nevyazka_hb = hb_predictor.predict(x_hb_train)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_hb_test, predicted_hb))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_hb_test, predicted_hb))
plt.scatter(y_hb_train, nevyazka_hb)
plt.scatter(y_hb_test, predicted_hb)

plt.show()

In [ ]:
importance = hb_predictor.coef_
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
x_hb_train.columns

In [ ]:
best_features = [
#    "Энергия деформирования Дж",	
#    "Энергия упругого деформирования Дж",	
#    "Энергия вязкого деформирования Дж",
#    "e",
    'Тангенс потерь град',
    'P / a', 
    'Pmax/apmax', 
    'Pamax/amax',
    "HB"
]
df_bf = df[best_features]
sns.pairplot(df_bf)
plt.show()

In [ ]:
y_df_b = df_bf["HB"]
x_df_b = df_bf.drop(columns="HB")

x_b_train, x_b_test, y_b_train, y_b_test = train_test_split(x_df_b, y_df_b, 
                                                train_size=0.8, 
                                                random_state=18)

In [ ]:
hb_predictor = LinearRegression(n_jobs=-1)
hb_predictor.fit(X=x_b_train, y=y_b_train)

predicted_hb = hb_predictor.predict(x_b_test)
nevyazka_hb = hb_predictor.predict(x_b_train)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_b_test, predicted_hb))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_b_test, predicted_hb))
plt.scatter(y_b_train, nevyazka_hb)
plt.scatter(y_b_test, predicted_hb)

plt.show()

In [ ]:
importance = hb_predictor.coef_
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()